In [1]:
import numpy as np
import cv2
import pickle
import glob
import matplotlib.pyplot as plt
from tracker import tracker
%matplotlib notebook

#  from tracker import tracker

#  read in the saved objpoints and imagepoints

dist_pickle = pickle.load(open("calibration_pickle.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):

    # Calculate gradient direction
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.cvtColor(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.cvtColor(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    with np.errstate(divide="ignore", invalid="ignore"):
        absgraddir = np.absolute(np.arctan(sobely/sobelx))
        binary_output = np.zeros_like(absgraddir)
        # Apply threshold
        binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return binary_output

In [2]:
def abs_sobel_thresh(image, orient='x', thresh=(0, 255)):
    # Convert to Grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    else:
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # create a copy and apply the threshold
    binary_output = np.zeros_like(gray)#scaled_sobel)
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    #sxbinary     [(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    # sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    #print("binary_output", binary_output)
    #preprocessImage = np.zeros_like(gray)#image[:, :, 0])
    #preprocessImage[(binary_output == 1)] = 255
    #plt.imshow(binary_output)
    #plt.title('window fitting results')
    #plt.show()

    #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    #f.tight_layout()
    ###ax1.imshow(image)
    #ax1.set_title('Original Image', fontsize=50)
    #ax2.imshow(scaled_sobel, cmap='gray')
    #ax2.set_title('Thresholded Magnitude', fontsize=50)
    #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    #plt.imshow(preprocessImage, cmap='gray')
    #cv2.imwrite('./sobeltest.jpg', binary_output)


    return binary_output# preprocessImage

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(image,  cv2.COLOR_RGB2GRAY)

    # take derivative in x and y direction
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the Gradient max
    gradmax = np.sqrt(sobelx**2 + sobely**2)
    # scale to 8bit (0-255) then convert to uint8
    scaled_sobel = np.uint8(255*gradmax/np.max(gradmax))

    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(sxbinary >= mag_thresh[0]) & (sxbinary <= mag_thresh[1])]
    return sxbinary

def color_threshold(image, sthresh=(0, 255), vthresh=(0, 255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)#.astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:, :, 2]
    #sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    #abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    #scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0]) & (s_channel <= sthresh[1])] = 1

    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype(np.float)
    v_channel = hsv[:, :, 2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= vthresh[0]) & (v_channel <= vthresh[1])] = 1
    #print("V", v_binary)

    output = np.zeros_like(s_channel)
    output[(s_binary == 1) & (v_binary == 1)] = 1
    return output
    

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

In [3]:
import matplotlib.pyplot as plt
def visualize(image):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(image, cmap='gray')
    ax2.set_title('Thresholded Magnitude', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [7]:
from tracker import tracker
images = glob.glob('./test_images/test*.jpg')
for idx, fname in enumerate(images):
    # read the image
    img = cv2.imread(fname)
    # undistort the images
    img = cv2.undistort(img, mtx, dist, None, mtx)
    # print(img)

    # process image and generate binary pixel of interest
    preprocessImage = np.zeros_like(img[:, :, 0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(12, 255)) #20, 120
    # print(gradx)
    grady = abs_sobel_thresh(img, orient='y', thresh=(25, 255)) #25, 150
    c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50, 255))#170,255
    #print(c_binary)
    preprocessImage[((gradx ==1)&(grady == 1))|(c_binary == 1)] = 255
    
    #Define perspective transform
    img_size = (img.shape[1], img.shape[0])
    bot_width = .76 #% of bottom trapeziodal width
    mid_width = .16   #% of middle trapezoidal width
    height_pct = .66 #% of trapezoidal height
    bottom_trim = .935 #% to avoid car hood
    src = np.float32([[img.shape[1]*(.5-mid_width/2), img.shape[0]*height_pct], 
                      [img.shape[1]*(.5+mid_width/2), img.shape[0]*height_pct],
                      [img.shape[1]*(.5+bot_width/2), img.shape[0]*bottom_trim], 
                      [img.shape[1]*(.5-bot_width/2), img.shape[0]*bottom_trim],])
    offset = img_size[0]*.25
    dst = np.float32([[offset, 0], 
                      [img_size[0]-offset, 0], 
                      [img_size[0]-offset, img_size[1]],
                      [offset, img_size[1]]])
    #print("src", src)
    #print("dst", dst)
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(preprocessImage, M, img_size, flags=cv2.INTER_LINEAR)
    window_width = 25
    window_height = 40
    #Set up the overall class to do all the tracking
    curve_centers = tracker(Mywindow_width = window_width, Mywindow_height = window_height, Mymargin = 25, My_ym = 10/720, My_xm = 4/384, Mysmooth_factor = 15)
    window_centroids = []
    window_centroids = curve_centers.find_window_centroids(warped)
    print(window_centroids)
    
    #Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
    
    
    rightx = []
    leftx = []
    
    
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        # Add center value found in frame to the list of lane points per left, right
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])

        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
        
    print("L point", l_points.shape)
    print("R point", l_points.shape)
    

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    #print("template", template.shape)
    zero_channel = np.zeros_like(template) # create a zero color channel
    #print("zero channel", zero_channel.shape)
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    #visualize(template)
    print("template2", template.shape)
    warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
    #print("warpage", warpage.shape)
    output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
    
    #Fit the lane boundaries to the left , right center positions found
    yvals = range(0, warped.shape[0])
    res_yvals = np.arange(warped.shape[0]-(window_height/2),0,-window_height)
    
    left_fit =np.polyfit(res_yvals, leftx, 2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
    left_fitx = np.array(left_fitx, np.int32)
    
    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    right_fitx = np.array(right_fitx, np.int32)
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,left_fitx[::-1]+window_width/2), axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2), axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    inner_lane = np.array(list(zip(np.concatenate((left_fitx+window_width/2,right_fitx[::-1]-window_width/2), axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    #curve_pts = np.array[list(zip(curve_fit)))
    
    road = np.zeros_like(img)
    cv2.fillPoly(road, [left_lane], color=[255,0,0])
    cv2.fillPoly(road, [right_lane], color=[0,0,255])
    
    
    
    
    result = road
    #print("result", result.shape)
    write_name = './test_images/tracked' + str(idx) + '.jpg'
    cv2.imwrite(write_name, result)
    #exit()

[[ 416.5  929.5]
 [ 417.5  926.5]
 [ 416.5  931.5]
 [ 413.5  933.5]
 [ 413.5  908.5]
 [ 414.5  883.5]
 [ 414.5  858.5]
 [ 414.5  833.5]
 [ 416.5  808.5]
 [ 413.5  783.5]
 [ 417.5  758.5]
 [ 417.5  733.5]
 [ 418.5  708.5]
 [ 416.5  683.5]
 [ 419.5  658.5]
 [ 415.5  633.5]
 [ 422.5  608.5]
 [ 423.5  583.5]]
L point (720, 1280)
R point (720, 1280)
template2 (720, 1280, 3)
[[ 441.5  910.5]
 [ 443.5  908.5]
 [ 442.5  906.5]
 [ 440.5  902.5]
 [ 438.5  902.5]
 [ 436.5  900.5]
 [ 434.5  875.5]
 [ 432.5  850.5]
 [ 936.5  825.5]
 [ 934.5  800.5]
 [ 932.5  775.5]
 [ 935.5  750.5]
 [ 939.5  725.5]
 [ 914.5  700.5]
 [ 889.5  675.5]
 [ 906.5  650.5]
 [ 915.5  625.5]
 [ 923.5  600.5]]
L point (720, 1280)
R point (720, 1280)
template2 (720, 1280, 3)
[[ 419.5  923.5]
 [ 417.5  921.5]
 [ 419.5  924.5]
 [ 421.5  926.5]
 [ 424.5  901.5]
 [ 426.5  876.5]
 [ 430.5  851.5]
 [ 433.5  826.5]
 [ 438.5  801.5]
 [ 440.5  776.5]
 [ 443.5  751.5]
 [ 446.5  726.5]
 [ 450.5  701.5]
 [ 454.5  676.5]
 [ 457.5  651.5]
 

In [ ]:
https://github.com/awbrown90/Advance-Lane-Finding/blob/master/main_image_gen.py